In [1]:
import requests, json, time, os
from collections import Counter
from IPython.core.display import display, HTML

In [2]:
pathToBattles = r"~\AppData\LocalLow\Sky Mavis\Axie Infinity\production-battleHistoriesState"
pathToBattles = os.path.expanduser(pathToBattles)

In [3]:
my = "0x3ac722d50232118a80b1d60ca787f4cd70e12813"
with open(pathToBattles, "rt") as f:
    data = json.load(f)
battles = data["battles"]

teams = {}
for battle in battles:
    if battle["battleType"] == 1:
        first = battle["firstClientId"]
        second = battle["secondClientId"]
        enemyNo = first == my and  1 or 0
        enemyId = first == my and second or first
        team = []
        for fighter in battle["fighters"]:
            if fighter["teamIndex"] == enemyNo:
                team.append(fighter["id"])
        teams[enemyId] = team

In [ ]:
for enemy, team in teams.items():
    print(enemy, team)

In [4]:
from python_graphql_client import GraphqlClient
client = GraphqlClient(endpoint="https://graphql-gateway.axieinfinity.com/graphql")

In [5]:
def getProfileData(address):
    query = """query GetProfileByRoninAddress($roninAddress: String!) {\n  publicProfileWithRoninAddress(roninAddress: $roninAddress) {\n    ...Profile\n    __typename\n  }\n}\n\nfragment Profile on PublicProfile {\n  accountId\n  name\n  addresses {\n    ...Addresses\n    __typename\n  }\n  __typename\n}\n\nfragment Addresses on NetAddresses {\n  ethereum\n  tomo\n  loom\n  ronin\n  __typename\n}\n"""
    variables = {"roninAddress": address}
    data = client.execute(query=query, variables=variables)
    return data

In [6]:
getProfileData("0x3ac722d50232118a80b1d60ca787f4cd70e12813")

{'data': {'publicProfileWithRoninAddress': {'accountId': '4836585',
   'name': 'pinkwonder',
   'addresses': {'ethereum': None,
    'tomo': None,
    'loom': '0x9b29c6a400cb7cdaf64dca893f895189fa491482',
    'ronin': '0x3ac722d50232118a80b1d60ca787f4cd70e12813',
    '__typename': 'NetAddresses'},
   '__typename': 'PublicProfile'}}}

In [ ]:
mmr

In [ ]:
mmr_addr = "https://game-api.axie.technology/mmr/"
addresses = list(teams.keys())
print(len(addresses))
answer = []
for i in range(len(addresses)//10+1):
    mmr_addrs = addresses[i*10:i*10+10]
    if len(mmr_addrs) > 0:
        mmr_req = mmr_addr + ",".join(mmr_addrs)
        print(mmr_req)
        #try:
        data = requests.get(mmr_req).json()
        for result in data:
            for item in result["items"]:
                if item["client_id"] in mmr_addrs:
                    client_id = item["client_id"]
                    elo = item["elo"]
                    rank = item["rank"]
                    name = item["name"]
                    answer.append([client_id, elo, rank, name])
    time.sleep(3.0)

98
https://game-api.axie.technology/mmr/0xf91bedeac9e3489aa34da14da0fdc3a58d74346c,0x587b154bf357759e646965d023d87e2cd16252a5,0xe1ea078bf1d01ef11e5f4e3bca5d38911f4b30e5,0x84ec9b3315acbacf34398d69d71c2047f2724168,0xefd3d32e75c1b731465bedac1db710c298f8c046,0x921aa67cae6f6b93997c0fd9af9dc04c8e428380,0xfed281e91609274eee69a640df1f335167d17ee0,0xca01ac28b6d9122b1153d84b3d0c87d9b96c265c,0xf1ee9c713df846ca628c42ea346c0364a154bf80,0xbc364814ed3fb0229a3364e0afd418e76b0b495a
https://game-api.axie.technology/mmr/0xcb1bfe298b252a4fe6343c610ab790fe151a0c31,0x8f2338a40e316d7a79a867b5a12c371871065466,0x5e73c9a0e25381fec17fa43430e0f6e4efcd880c,0x93af629f2b9e097cecd26bf29f9fd49b91974a45,0x0d49809ad057f29db8c36a400ca7e7b5d690bc1d,0x4771de1c20809fbc465a087597426933f01b6d08,0xf8588e7d57fb6f9ee95f1999e357541c57a344df,0xee11cdbee359ad0e212239d3830d4e9c09efcc29,0x41a09f2b276744f6cb3423c91af5786ac5e0251c,0x8d0c79fd5231846f37699e16e336e144d7121548


In [ ]:
answer.sort(key = lambda x: -x[1])

In [ ]:
example_link = "https://marketplace.axieinfinity.com/axie/"
def displayAxies(axieList, axieImages, displayLink):
    lines = []
    for axie, img in zip(axieList, axieImages):
        link = example_link + str(axie)
        if displayLink:
            print(link)
        lines.append('<a href="%s" target="_blank" rel="noopener noreferrer"><img src="%s" width=128 height=128 style="display:inline; margin:1px"/></a>'%(link,img))
    display(HTML("".join(lines)))

In [ ]:
link_templ = "https://marketplace.axieinfinity.com/profile/ronin:%s/axie/"
for client_id, elo, rank, name in answer:
    link_id = link_templ%client_id[2:]
    print(link_id, elo, rank, name)
    axieList = teams[client_id]
    axieImages = ["https://storage.googleapis.com/assets.axieinfinity.com/axies/%s/axie/axie-full-transparent.png"%x for x in axieList]
    displayAxies(axieList, axieImages, False)